In [1]:
from pathlib import Path
import xarray as xr
import pandas as pd
import numpy as np

from functions import *

data_dir = Path("/beegfs/CMIP6/jdpaul3/hydroviz_data/stats")

### Split & filter 
Split the collection of CSVs into two file groups based on geometry (`seg` = stream segment polyline geometry, and `hru` = hydrological response unit geometry, ie watershed). Filter out the files we don't want (the Maurer files, the "diff" files, and any files with stats computed over the full historical range [1952-2005].)

In [2]:
seg_files = list(data_dir.glob("dynamic*seg*.csv"))
seg_files += list(data_dir.glob("static*seg*.csv"))

hru_files = list(data_dir.glob("dynamic*hru*.csv"))
hru_files += list(data_dir.glob("static*hru*.csv"))

In [3]:
seg_files = filter_files(seg_files)
hru_files = filter_files(hru_files)

Removed 317 files from list; 314 files remain out of 631 original files.
Removed 317 files from list; 314 files remain out of 631 original files.


### Extract statistic names, geometry IDs, and coordinates

Get the stats abbreviations from columns of the first file; we will use these as variables when building our xarray dataset. Also get the geometry ids from the index of the first files in each list; we will use these as coordinates in our `xarray` datasets.

In [4]:
stat_vars = pd.read_csv(seg_files[0]).columns[1:].tolist()

In [5]:
seg_ids = pd.read_csv(seg_files[0]).seg_id.astype(str).tolist()
hru_ids = pd.read_csv(hru_files[0]).hru_id.astype(str).tolist()

In [6]:
geom_coords_dict = {}
geom_coords_dict["seg"] = get_unique_coords(seg_files)
geom_coords_dict["hru"] = get_unique_coords(hru_files)
geom_coords_dict

{'seg': {'lcs': ['dynamic', 'static'],
  'models': ['ACCESS1-0',
   'BNU-ESM',
   'CCSM4',
   'GFDL-ESM2G',
   'GFDL-ESM2M',
   'IPSL-CM5A-LR',
   'IPSL-CM5A-MR',
   'MIROC-ESM',
   'MIROC-ESM-CHEM',
   'MIROC5',
   'MRI-CGCM3',
   'NorESM1-M',
   'bcc-csm1-1'],
  'scenarios': ['historical', 'rcp26', 'rcp45', 'rcp60', 'rcp85'],
  'variants': ['r1i1p1'],
  'eras': ['1976_2005', '2016_2045', '2046_2075', '2071_2100']},
 'hru': {'lcs': ['dynamic', 'static'],
  'models': ['ACCESS1-0',
   'BNU-ESM',
   'CCSM4',
   'GFDL-ESM2G',
   'GFDL-ESM2M',
   'IPSL-CM5A-LR',
   'IPSL-CM5A-MR',
   'MIROC-ESM',
   'MIROC-ESM-CHEM',
   'MIROC5',
   'MRI-CGCM3',
   'NorESM1-M',
   'bcc-csm1-1'],
  'scenarios': ['historical', 'rcp26', 'rcp45', 'rcp60', 'rcp85'],
  'variants': ['r1i1p1'],
  'eras': ['1976_2005', '2016_2045', '2046_2075', '2071_2100']}}

Besides the geometry, the coords are equivalent. We won't use the `variant` coord since there is only one variant represented here.

In [7]:
# seg_ds = create_empty_dataset(geom_coords_dict["seg"], stat_vars, seg_ids)
# seg_ds

In [8]:
# hru_ds = create_empty_dataset(geom_coords_dict["hru"], stat_vars, hru_ids)
# hru_ds

In [9]:
len(hru_ids)

109951

### Populate empty NC files

In [10]:
seg_ds = xr.open_dataset("/beegfs/CMIP6/jdpaul3/hydroviz/empty_seg.nc")
#hru_ds = xr.open_dataset("/beegfs/CMIP6/jdpaul3/hydroviz/empty_hru.nc")

In [11]:
seg_ds

<xarray.Dataset> Size: 12GB
Dimensions:   (lc: 2, model: 13, scenario: 5, era: 4, geom_id: 56540)
Coordinates:
  * lc        (lc) <U7 56B 'dynamic' 'static'
  * model     (model) <U14 728B 'ACCESS1-0' 'BNU-ESM' ... 'bcc-csm1-1'
  * scenario  (scenario) <U10 200B 'historical' 'rcp26' 'rcp45' 'rcp60' 'rcp85'
  * era       (era) <U13 208B 'historical' 'early_century' ... 'late_century'
  * geom_id   (geom_id) <U5 1MB '1' '2' '3' '4' ... '56538' '56539' '56540'
Data variables: (12/52)
    ma3       (lc, model, scenario, era, geom_id) float64 235MB ...
    ma4       (lc, model, scenario, era, geom_id) float64 235MB ...
    ma12      (lc, model, scenario, era, geom_id) float64 235MB ...
    ma13      (lc, model, scenario, era, geom_id) float64 235MB ...
    ma14      (lc, model, scenario, era, geom_id) float64 235MB ...
    ma15      (lc, model, scenario, era, geom_id) float64 235MB ...
    ...        ...
    sum_dur3  (lc, model, scenario, era, geom_id) float64 235MB ...
    sum_dur7  (lc, model, scenario, era, geom_id) float64 235MB ...
    sum_freq  (lc, model, scenario, era, geom_id) float64 235MB ...
    sum_ord   (lc, model, scenario, era, geom_id) float64 235MB ...
    sum_cv    (lc, model, scenario, era, geom_id) float64 235MB ...
    lf1       (lc, model, scenario, era, geom_id) float64 235MB ...

In [12]:
def populate_dataset(ds, files):
    # add a step to confirm that the CSV is the proper shape based on dataset geom_id coords
    # add a step to confirm that the CSV column names are identical to the dataset stat_vars
    # add a step to confirm that the parsed coords all actually exist in the dataset

    for file in files:
        # parse this filename to find coords where data should go
        parts = file.name.split('_')
        try:
            lc, model, scenario, era = parts[0], parts[1], parts[2], "_".join([parts[5], parts[6].split(".")[0]])
        except:
            print(f"Error parsing file: {file.name}")
            continue
        
        df = pd.read_csv(file)
        df.replace(-99999, np.nan, inplace=True)

        for stat in df.columns[1:]:
            ds[stat].loc[{"lc": lc, "model": model, "scenario": scenario, "era": era_lookup[era]}] = df[stat]
        


In [13]:
populate_dataset(seg_ds, seg_files[0:5])

In [ ]:
seg_ds["ma3"].sel({"lc": "dynamic", "model": "GFDL-ESM2M", "scenario": "rcp60", "era": "mid_century"}).load()

In [28]:
# parse this filename to find coords where data should go
seg_files[0]

PosixPath('/beegfs/CMIP6/jdpaul3/hydroviz_data/stats/dynamic_GFDL-ESM2M_rcp60_r1i1p1_seg_2046_2075.csv')

In [29]:
# replace the -99999 with np.nan
# set index to geom_id
# and for each variable, set the data in the xarray dataset along the geom_id dimension
df = pd.read_csv(seg_files[0])
df = df.replace(-99999, np.nan)
df

,seg_id,ma3,ma4,ma12,ma13,ma14,ma15,ma16,ma17,ma18,...,spr_dur7,spr_freq,spr_ord,sum_mag,sum_dur3,sum_dur7,sum_freq,sum_ord,sum_cv,lf1
0,1,192.27,147.78,64.59,19.182,53.315,444.23,240.61,101.62,73.375,...,1397.8,9.5,111.5,0.020236,1.5913,5.7061,7.5,256.5,0.996485,71.0
1,2,117.47,100.76,389.30,328.330,595.210,1147.90,479.31,288.17,180.410,...,3302.6,6.5,103.0,0.015262,3.1210,3.8311,7.5,261.5,0.835755,23.5
2,3,145.66,122.20,704.50,444.750,856.800,3208.40,1265.20,597.88,399.090,...,9258.7,9.0,109.0,0.008106,5.5370,10.9130,4.0,258.0,0.795020,35.5
3,4,141.67,119.15,760.04,500.360,932.200,3346.60,1338.10,633.36,419.520,...,9590.6,9.0,109.0,0.008157,6.1173,12.5420,4.5,258.0,0.776000,29.0
4,5,116.26,100.93,456.57,398.950,729.260,1378.10,569.92,340.81,212.900,...,3901.6,6.5,103.0,0.014536,3.5317,4.4323,7.5,262.0,0.768025,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56535,56536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56536,56537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56537,56538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56538,56539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# for instance, see what the values are for the first stat ma3 (should be all NAN because its empty)
seg_ds["ma3"].sel({"lc": "dynamic", "model": "GFDL-ESM2M", "scenario": "rcp60", "era": "mid_century"}).load()

<xarray.DataArray 'ma3' (geom_id: 56540)> Size: 452kB
array([nan, nan, nan, ..., nan, nan, nan])
Coordinates:
    lc        <U7 28B 'dynamic'
    model     <U14 56B 'GFDL-ESM2M'
    scenario  <U10 40B 'rcp60'
    era       <U13 52B 'mid_century'
  * geom_id   (geom_id) <U5 1MB '1' '2' '3' '4' ... '56538' '56539' '56540'

In [32]:
#now use indexing (instead of sel or isel) to set new values
seg_ds["ma3"].loc[{"lc": "dynamic", "model": "GFDL-ESM2M", "scenario": "rcp60", "era": "mid_century"}] = 1

In [33]:
# view again to be sure they are set
seg_ds["ma3"].sel({"lc": "dynamic", "model": "GFDL-ESM2M", "scenario": "rcp60", "era": "mid_century"}).load()

<xarray.DataArray 'ma3' (geom_id: 56540)> Size: 452kB
array([1., 1., 1., ..., 1., 1., 1.])
Coordinates:
    lc        <U7 28B 'dynamic'
    model     <U14 56B 'GFDL-ESM2M'
    scenario  <U10 40B 'rcp60'
    era       <U13 52B 'mid_century'
  * geom_id   (geom_id) <U5 1MB '1' '2' '3' '4' ... '56538' '56539' '56540'

In [34]:
#lets try with real values now
seg_ds["ma3"].loc[{"lc": "dynamic", "model": "GFDL-ESM2M", "scenario": "rcp60", "era": "mid_century"}] = df['ma3']

In [35]:
seg_ds["ma3"].sel({"lc": "dynamic", "model": "GFDL-ESM2M", "scenario": "rcp60", "era": "mid_century"}).load()

<xarray.DataArray 'ma3' (geom_id: 56540)> Size: 452kB
array([192.27, 117.47, 145.66, ...,    nan,    nan,    nan])
Coordinates:
    lc        <U7 28B 'dynamic'
    model     <U14 56B 'GFDL-ESM2M'
    scenario  <U10 40B 'rcp60'
    era       <U13 52B 'mid_century'
  * geom_id   (geom_id) <U5 1MB '1' '2' '3' '4' ... '56538' '56539' '56540'